Environment Setup


In [ ]:
# Importing libraries
import pandas as pd
import math
import numpy as np
import os
print('Libraries Imported')

#!unzip yelp_reviews.zip
df= pd.read_csv('police.csv')
print(df.head())

print('Setup Complete')

Libraries Imported
        Incident Datetime  ... Current Police Districts
0  2020/03/11 09:10:00 AM  ...                      NaN
1  2018/07/04 08:00:00 AM  ...                      NaN
2  2020/01/31 04:00:00 AM  ...                      NaN
3  2020/02/16 09:00:00 AM  ...                      NaN
4  2020/02/20 01:15:00 PM  ...                      NaN

[5 rows x 16 columns]
Setup Complete


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['police.csv']))
df

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime, time
le = LabelEncoder()


df["Incident Day of Week"] =  le.fit_transform(df["Incident Day of Week"])
#df['Incident_Category'] = le.fit_transform(df['Incident_Category'].astype(str))
df["Analysis Neighborhood"] =  le.fit_transform(df["Analysis Neighborhood"].astype(str))
df["Police District"] =  le.fit_transform(df["Police District"])

df.point.str.extract(r'\((?P<lat>[\-\d\.]+),\s+(?P<lon>[\-\d\.]+)\)')
# df2['combined'] = df2[['Latitude', 'Longitude']].apply(list, axis=1)
# df2['combined']= df2.combined.astype(float)
df['Incident Time']= pd.to_datetime(df['Incident Time'])

df['period'] = (df['Incident Time'].dt.hour % 24 + 4) // 4
df['period'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)
df["period"] =  le.fit_transform(df["period"])

df

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Incident_Category,Incident Description,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point,SF Find Neighborhoods,Current Police Districts,period
0,2020/03/11 09:10:00 AM,2020/03/11,2020-10-29 09:10:00,2020,6,2020/03/11 09:10:00 AM,Recovered Vehicle,"Vehicle, Recovered, Truck",5,41,NaN,NaN,NaN,NaN,NaN,NaN,3
1,2018/07/04 08:00:00 AM,2018/07/04,2020-10-29 08:00:00,2018,6,2018/07/13 09:31:00 AM,Lost Property,Lost Property,5,41,NaN,NaN,NaN,NaN,NaN,NaN,3
2,2020/01/31 04:00:00 AM,2020/01/31,2020-10-29 04:00:00,2020,0,2020/03/02 05:46:00 PM,Lost Property,Lost Property,5,41,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2020/02/16 09:00:00 AM,2020/02/16,2020-10-29 09:00:00,2020,3,2020/02/19 08:21:00 AM,Larceny Theft,"Theft, Other Property, >$950",5,41,NaN,NaN,NaN,NaN,NaN,NaN,3
4,2020/02/20 01:15:00 PM,2020/02/20,2020-10-29 13:15:00,2020,4,2020/02/24 06:26:00 PM,Lost Property,Lost Property,5,41,NaN,NaN,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65529,2020/06/22 04:54:00 PM,2020/06/22,2020-10-29 16:54:00,2020,1,2020/06/22 04:54:00 PM,Fraud,False Personation,5,19,6.0,37.773467,-122.391434,"(37.773466920607476, -122.39143433652146)",34.0,1.0,1
65530,2020/06/22 04:54:00 PM,2020/06/22,2020-10-29 16:54:00,2020,1,2020/06/22 04:54:00 PM,Fraud,"Credit Application, Fraudulent",5,19,6.0,37.773467,-122.391434,"(37.773466920607476, -122.39143433652146)",34.0,1.0,1
65531,2020/06/14 03:07:00 PM,2020/06/14,2020-10-29 15:07:00,2020,3,2020/06/14 03:07:00 PM,Fraud,"Theft, False Pretenses, Felony",5,19,6.0,37.773467,-122.391434,"(37.773466920607476, -122.39143433652146)",34.0,1.0,5
65532,2020/01/06 10:00:00 AM,2020/01/06,2020-10-29 10:00:00,2020,1,2020/01/14 02:05:00 AM,Larceny Theft,"Theft, From Locked Vehicle, >$950",7,41,NaN,NaN,NaN,NaN,NaN,NaN,3


In [ ]:
len(df['Incident Year'].unique())

3

In [ ]:
!pip install -U folium
!pip install -U geopandas

Requirement already up-to-date: folium in /usr/local/lib/python3.6/dist-packages (0.11.0)
Requirement already up-to-date: geopandas in /usr/local/lib/python3.6/dist-packages (0.8.1)


In [ ]:
import folium
import geopandas

In [ ]:
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame

# from folium.plugins import HeatMap
# map1 = folium.Map(location=(37.76, -122.45), zoom_start=11)
# shape = geopandas.read_file('geo_export_e21612ee-1948-49ab-a925-adaad6bfb384.geojson')
# shape.to_file('Voting_Wards.geojson', driver='GeoJSON')

# folium.GeoJson(
#    shape,
# ).add_to(map1)

df2 = df[(df['Incident_Category']=='Homicide') | (df['Incident_Category']=="Prostitution") |
       (df['Incident_Category']=="Rape") |
         (df['Incident_Category']=="Suicide") |
       (df['Incident_Category']=="Sex Offense")
        |
       (df['Incident_Category']=="Family Offense")]
df2 = df2.dropna()
lat=list(df2["Latitude"])
lon=list(df2["Longitude"])
crimeType = list(df2["Incident_Category"])
crimeDescr = list(df2["Incident Description"])

for lt,ln,crm, desc in zip(lat,lon, crimeType, crimeDescr):
    if crm == 'Homicide':
        popup = crm
        icon = folium.Icon(color="black", icon="fire")
    elif crm == 'Prostitution':
        popup = crm
        icon = folium.Icon(color="orange", icon="fire")
    elif crm == 'Rape':
        popup = crm
        icon = folium.Icon(color="green", icon="fire")
    elif crm == 'Family Offense':
        popup = crm, desc
        icon = folium.Icon(color="darkgreen", icon="fire")
    elif crm == 'Suicide':
        icon = folium.Icon(color="blue", icon="fire")
        popup = crm
    elif crm == 'Sex Offense':
        if desc == 'Child Abuse, Pornography': 
            icon = folium.Icon(color="purple", icon="fire")
            popup = crm, desc
        else: 
            icon = folium.Icon(color="darkpurple", icon="fire")
            popup = crm, desc
    folium.Marker(location=[lt,ln], icon= icon).add_to(map1)  



folium.LayerControl().add_to(map1)



map1

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime, time
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error
dfFamilyOffense = df[df['Incident_Category']=="Family Offense"]
X_trainfamLat, X_testfamLat, y_trainfamLat, y_testfamLat =  train_test_split(dfFamilyOffense, dfFamilyOffense.Latitude, test_size=0.2)

myFeatsTrainLat = X_trainfamLat[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]
myFeatsTestLat = X_testfamLat[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]

IncidentEncodedTrainLat = (X_trainfamLat["Latitude"])
IncidentEncodedTestLat = (X_testfamLat["Latitude"])

clffam = MLPRegressor(hidden_layer_sizes=(10), learning_rate = 'adaptive', activation='tanh', 
                     max_iter=333, solver='lbfgs', verbose=1, random_state=42)

clffam.fit(myFeatsTrainLat, IncidentEncodedTrainLat)

print('Mean Squared Error on training LAT---')
y_pred_trainfam1 = clffam.predict(myFeatsTrainLat)
print(mean_squared_error(IncidentEncodedTrainLat, y_pred_trainfam1))

print('Mean Squared Error on test LAT---')
y_pred_testfam1=clffam.predict(myFeatsTestLat)
print(mean_squared_error(IncidentEncodedTestLat, y_pred_testfam1))


print('Mean Absolute Error on training LAT---')
y_pred_trainfam2 = clffam.predict(myFeatsTrainLat)
print(mean_absolute_error(IncidentEncodedTrainLat, y_pred_trainfam2))

print('Mean Absolute Error on test LAT---')
y_pred_testfam2 = clffam.predict(myFeatsTestLat)
print(mean_absolute_error(IncidentEncodedTestLat, y_pred_testfam2))

#I split up the lat and long bc if i predicted them together it worsened my predictions

X_trainfamLon, X_testfamLon, y_trainfamLon, y_testfamLon =  train_test_split(dfFamilyOffense, dfFamilyOffense.Longitude, test_size=0.2)

myFeatsTrainLon = X_trainfamLon[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]
myFeatsTestLon = X_testfamLon[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]

IncidentEncodedTrainLon = (X_trainfamLon[ "Longitude"])
IncidentEncodedTestLon = (X_testfamLon["Longitude"])

clffam2 = MLPRegressor(hidden_layer_sizes=(10), learning_rate = 'adaptive', activation='tanh', 
                     max_iter=333, solver='lbfgs', verbose=1, random_state=42)

clffam2.fit(myFeatsTrainLon, IncidentEncodedTrainLon)

print('Mean Squared Error on training LON---')
y_pred_trainfam3=clffam2.predict(myFeatsTrainLon)
print(mean_squared_error(IncidentEncodedTrainLon, y_pred_trainfam3))

print('Mean Squared Error on test LON---')
y_pred_testfam3=clffam2.predict(myFeatsTestLon)
print(mean_squared_error(IncidentEncodedTestLon, y_pred_testfam3))


print('Mean Absolute Error on training LON---')
y_pred_trainfam4=clffam2.predict(myFeatsTrainLon)
print(mean_absolute_error(IncidentEncodedTrainLon, y_pred_trainfam4))

print('Mean Absolute Error on test LON---')
y_pred_testfam4=clffam2.predict(myFeatsTestLon)
print(mean_absolute_error(IncidentEncodedTestLon, y_pred_testfam4))
#I need to try rescaling the lat/long data which explains why the error is so low because lat and long are very minutely different but make a huge difference when you map it out

Mean Squared Error on training LAT---
0.000610488154537926
Mean Squared Error on test LAT---
0.0005888787226001794
Mean Absolute Error on training LAT---
0.020891483797640737
Mean Absolute Error on test LAT---
0.020232505809807913
Mean Squared Error on training LON---
0.0006479754784251898
Mean Squared Error on test LON---
0.0009497845170039204
Mean Absolute Error on training LON---
0.019086340094264045
Mean Absolute Error on test LON---
0.021846240662100058


In [ ]:

dffam=  {'lat':y_pred_trainfam1 , 'long': y_pred_trainfam3}
dffam = pd.DataFrame(dffam)
df_fam = df[df["Incident_Category"] == "Family Offense"]
mapfam = folium.Map(location=(37.76, -122.45), zoom_start=11)

lat=list(dffam['lat'])
lon=list(dffam['long'])

for lt,ln in zip(lat,lon):
    folium.Marker(location=[lt,ln]).add_to(mapfam)  

lat_ = list(df_fam['Latitude'])
lon_ = list(df_fam['Longitude'])


for lt,ln in zip(lat_,lon_):
    folium.Marker(location=[lt,ln], icon = folium.Icon(color="darkpurple", icon="fire")).add_to(mapfam)  
mapfam

In [ ]:
import math

def latlon_to_xyz(lat,lon):
    """Convert angluar to cartesian coordiantes

    latitude is the 90deg - zenith angle in range [-90;90]
    lonitude is the azimuthal angle in range [-180;180] 
    """
    r = 6371 # https://en.wikipedia.org/wiki/Earth_radius
    theta = np.pi/2 - np.radians(lat) 
    phi = np.radians(lon)
    x = r * np.sin(theta) * np.cos(phi) # bronstein (3.381a)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    return [x,y,z]

df['x'],df['y'], df['z'] = latlon_to_xyz(df.Latitude, df.Longitude)

In [ ]:
df

,Incident Datetime,Incident Date,Incident Time,Incident Year,Incident Day of Week,Report Datetime,Incident_Category,Incident Description,Police District,Analysis Neighborhood,Supervisor District,Latitude,Longitude,point,SF Find Neighborhoods,Current Police Districts,period,x,y,z
0,2020/03/11 09:10:00 AM,2020/03/11,2020-10-29 09:10:00,2020,6,2020/03/11 09:10:00 AM,Recovered Vehicle,"Vehicle, Recovered, Truck",5,41,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
1,2018/07/04 08:00:00 AM,2018/07/04,2020-10-29 08:00:00,2018,6,2018/07/13 09:31:00 AM,Lost Property,Lost Property,5,41,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
2,2020/01/31 04:00:00 AM,2020/01/31,2020-10-29 04:00:00,2020,0,2020/03/02 05:46:00 PM,Lost Property,Lost Property,5,41,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
3,2020/02/16 09:00:00 AM,2020/02/16,2020-10-29 09:00:00,2020,3,2020/02/19 08:21:00 AM,Larceny Theft,"Theft, Other Property, >$950",5,41,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
4,2020/02/20 01:15:00 PM,2020/02/20,2020-10-29 13:15:00,2020,4,2020/02/24 06:26:00 PM,Lost Property,Lost Property,5,41,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65529,2020/06/22 04:54:00 PM,2020/06/22,2020-10-29 16:54:00,2020,1,2020/06/22 04:54:00 PM,Fraud,False Personation,5,19,6.0,37.773467,-122.391434,"(37.773466920607476, -122.39143433652146)",34.0,1.0,1,-2697.726606,-4252.341969,3902.499191
65530,2020/06/22 04:54:00 PM,2020/06/22,2020-10-29 16:54:00,2020,1,2020/06/22 04:54:00 PM,Fraud,"Credit Application, Fraudulent",5,19,6.0,37.773467,-122.391434,"(37.773466920607476, -122.39143433652146)",34.0,1.0,1,-2697.726606,-4252.341969,3902.499191
65531,2020/06/14 03:07:00 PM,2020/06/14,2020-10-29 15:07:00,2020,3,2020/06/14 03:07:00 PM,Fraud,"Theft, False Pretenses, Felony",5,19,6.0,37.773467,-122.391434,"(37.773466920607476, -122.39143433652146)",34.0,1.0,5,-2697.726606,-4252.341969,3902.499191
65532,2020/01/06 10:00:00 AM,2020/01/06,2020-10-29 10:00:00,2020,1,2020/01/14 02:05:00 AM,Larceny Theft,"Theft, From Locked Vehicle, >$950",7,41,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from datetime import datetime, time
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error


dfFamilyOffense = df[df['Incident_Category']=="Family Offense"]

dfFamilyOffense.Latitude = dfFamilyOffense.Latitude* (math.pi / 180.0)

dfFamilyOffense.Longitude = dfFamilyOffense.Longitude* (math.pi / 180.0)

X_trainfamLat, X_testfamLat, y_trainfamLat, y_testfamLat =  train_test_split(dfFamilyOffense, dfFamilyOffense.Latitude, test_size=0.2)

myFeatsTrainLat = X_trainfamLat[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]
myFeatsTestLat = X_testfamLat[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]

IncidentEncodedTrainLat = (X_trainfamLat["Latitude"])
IncidentEncodedTestLat = (X_testfamLat["Latitude"])

clffam = MLPRegressor(hidden_layer_sizes=(10), learning_rate = 'adaptive', activation='tanh', 
                     max_iter=333, solver='lbfgs', verbose=1, random_state=42)

clffam.fit(myFeatsTrainLat, IncidentEncodedTrainLat)

print('Mean Squared Error on training LAT---')
y_pred_trainfam1 = clffam.predict(myFeatsTrainLat)
print(mean_squared_error(IncidentEncodedTrainLat, y_pred_trainfam1))

print('Mean Squared Error on test LAT---')
y_pred_testfam1=clffam.predict(myFeatsTestLat)
print(mean_squared_error(IncidentEncodedTestLat, y_pred_testfam1))


print('Mean Absolute Error on training LAT---')
y_pred_trainfam2 = clffam.predict(myFeatsTrainLat)
print(mean_absolute_error(IncidentEncodedTrainLat, y_pred_trainfam2))

print('Mean Absolute Error on test LAT---')
y_pred_testfam2 = clffam.predict(myFeatsTestLat)
print(mean_absolute_error(IncidentEncodedTestLat, y_pred_testfam2))

#I split up the lat and long bc if i predicted them together it worsened my predictions

X_trainfamLon, X_testfamLon, y_trainfamLon, y_testfamLon =  train_test_split(dfFamilyOffense, dfFamilyOffense.Longitude, test_size=0.2)

myFeatsTrainLon = X_trainfamLon[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]
myFeatsTestLon = X_testfamLon[['Police District','Supervisor District', 'Incident Day of Week', "period", "Analysis Neighborhood"]]

IncidentEncodedTrainLon = (X_trainfamLon[ "Longitude"])
IncidentEncodedTestLon = (X_testfamLon["Longitude"])

clffam2 = MLPRegressor(hidden_layer_sizes=(10), learning_rate = 'adaptive', activation='tanh', 
                     max_iter=333, solver='lbfgs', verbose=1, random_state=42)

clffam2.fit(myFeatsTrainLon, IncidentEncodedTrainLon)

print('Mean Squared Error on training LON---')
y_pred_trainfam3=clffam2.predict(myFeatsTrainLon)
print(mean_squared_error(IncidentEncodedTrainLon, y_pred_trainfam3))

print('Mean Squared Error on test LON---')
y_pred_testfam3=clffam2.predict(myFeatsTestLon)
print(mean_squared_error(IncidentEncodedTestLon, y_pred_testfam3))


print('Mean Absolute Error on training LON---')
y_pred_trainfam4=clffam2.predict(myFeatsTrainLon)
print(mean_absolute_error(IncidentEncodedTrainLon, y_pred_trainfam4))

print('Mean Absolute Error on test LON---')
y_pred_testfam4=clffam2.predict(myFeatsTestLon)
print(mean_absolute_error(IncidentEncodedTestLon, y_pred_testfam4))

Mean Squared Error on training LAT---
2.6557455889937183e-06
Mean Squared Error on test LAT---
5.729487632414668e-06
Mean Absolute Error on training LAT---
0.0009136212112482459
Mean Absolute Error on test LAT---
0.0012488456416909386
Mean Squared Error on training LON---
1.2349252845100289e-06
Mean Squared Error on test LON---
9.756657017384353e-07
Mean Absolute Error on training LON---
0.0008569661207074206
Mean Absolute Error on test LON---
0.000856136226070801


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
import math 

dfXYZ=  {'lat':y_pred_trainfam2, 'long': y_pred_trainfam4}


In [ ]:
dfLatLongfromxy

,lat,long
0,-41.879448,-122.429073
1,-41.880618,-122.414963
2,-41.852885,-122.442716
3,-41.849334,-122.418365
4,-41.890125,-122.423914
...,...,...
71,-41.891215,-122.423361
72,-41.821296,-122.458660
73,-41.869797,-122.410511
74,-41.866566,-122.435805


In [ ]:
dffam

,lat,long
0,37.739272,-122.209881
1,37.755334,-122.276731
2,37.912305,-122.451369
3,37.905219,-122.451369
4,37.752634,-122.450828
...,...,...
71,37.725181,-122.451369
72,37.800156,-122.273489
73,37.753024,-122.437224
74,37.753138,-122.451369


In [ ]:

dffam=  {'lat':y_pred_trainfam1 , 'long': y_pred_trainfam3}
dffam = pd.DataFrame(dffam)
dffam.lat = dffam.lat * 180/ (math.pi)
dffam['long']= dffam['long'] * 180/ (math.pi)

df_fam = df[df["Incident_Category"] == "Family Offense"]
mapfam = folium.Map(location=(37.76, -122.45), zoom_start=11)

lat=list(dffam['lat'])
lon=list(dffam['long'])

for lt,ln in zip(lat,lon):
    folium.Marker(location=[lt,ln]).add_to(mapfam)  

lat_ = list(df_fam['Latitude'])
lon_ = list(df_fam['Longitude'])


for lt,ln in zip(lat_,lon_):
    folium.Marker(location=[lt,ln], icon = folium.Icon(color="darkpurple", icon="fire")).add_to(mapfam)  
mapfam